In [ ]:
from pathlib import Path

import pandas as pd
import polars as pl
from util_func import *
from xlsxwriter.utility import xl_rowcol_to_cell

In [ ]:
survey_processed_dir = Path(
    "Q:/Data/Surveys/HouseholdSurveys/MTC-SFCTA2022/Processed_20240329"
)
hh_raw_filepath = survey_processed_dir / "hh.csv"
taz_spatial_join_dir = survey_processed_dir / "01-taz_spatial_join"
reformat_dir = survey_processed_dir / "02-reformat"
person_reformat_filepath = reformat_dir / "person-reformat.csv"
tour_extract_wkday_dir = survey_processed_dir / "03-tour_extract" / "wt_wkday"
tour_extract_allwk_dir = survey_processed_dir / "03-tour_extract" / "wt_7day"
out_dir = survey_processed_dir / "04b-summary_notebooks"
out_dir.mkdir(exist_ok=True)
out_filepath = out_dir / "02_AllTrips_Purpose.xlsx"

In [ ]:
def load_hh(hh_raw_filepath):
    """load raw hh file to get income

    easier to use the raw file to get the recoded broad income categories,
    rather than deal with the middle-of-the-category income values from 2a reformat
    """
    return pl.read_csv(hh_raw_filepath, columns=["hh_id", "income_broad"]).rename(
        {"hh_id": "hhno", "income_broad": "hh_income_broad_cat"}
    )


def load_person(person_reformat_filepath):
    return pl.read_csv(
        person_reformat_filepath,
        columns=["hhno", "pno", "pgend", "pagey"],
        # reformat did not pull in race/ethnicity
        # TODO parse race/ethnicity from person-taz_spatial_join
        # or pre-do it in reformat step
    )


def load_trip(trip_assign_day_filepath, hh, person):
    return (
        pl.read_csv(trip_assign_day_filepath)
        .join(hh, on="hhno", how="left")
        .join(person, on=["hhno", "pno"], how="left")
        .with_columns(
            # NOTE deptm is NOT using standard Daysim definitions, see 02/a-reformat.py
            # NOTE depart_hour was available in the raw data;
            #      but we removed that column in 02/a-reformat.py
            depart_hour=(pl.col("deptm") // 100),
            count=pl.lit(
                1
            ),  # CH 20240508: unsure if needed, copied over from 2019 code
        )
        .filter(
            (pl.col("trexpfac") > 0)  # has to do with weights
            & (pl.col("mode") > 0)  # mode is not other (0)
            & (pl.col("otaz") > 0)  # otaz must exist (i.e. not -1)
            & (pl.col("dtaz") > 0)  # dtaz must exist (i.e. not -1)
        )
        .to_pandas()
    )

In [ ]:
hh = load_hh(hh_raw_filepath)
person = load_person(person_reformat_filepath)

In [ ]:
# the output is sorted by the labels list
col_dict = {
    "dpurp": {
        "desc": "DPurp",
        "col": "dpurp",
        "vals": range(0, 8),
        "labels": [
            "0_Home",
            "1_Work",
            "2_School",
            "3_Escort",
            "4_PersBus",
            "5_Shop",
            "6_Meal",
            "7_SocRec",
        ],
    },
    "tod": {
        "desc": "TOD",
        "col": "depart_hour",
        "vals": range(0, 24),
        "labels": [f"{i:02}h" for i in range(0, 24)],
    },
    "day": {
        "desc": "DOW",
        "col": "day",
        "vals": range(1, 8),
        "labels": ["1_Mon", "2_Tue", "3_Wed", "4_Thu", "5_Fri", "6_Sat", "7_Sun"],
    },
    # TODO raceeth needs to be pulled in from person-taz_spatial_kpom
    # "raceeth": {
    #     "desc": "RaceEth",
    #     "col": "raceeth",
    #     "vals": range(1, 6),
    #     "labels": ["1_Hispanic", "2_Black", "3_Asian/PI", "4_White", "5_Other"],
    # },
    "hh_income_broad_cat": {
        "desc": "HHIncome",
        "col": "hh_income_broad_cat",  # recoded broad categories
        "vals": list(range(1, 7)) + [995, 999],
        "labels": [
            "000-25k",
            "025-50k",
            "050-75k",
            "075-100k",
            "100-200k",
            "200k+",
            "missing response",
            "prefer not to answer",
        ],
    },
    "age": {
        "desc": "Age",
        "col": "pagey",
        "vals": [3, 10, 16, 21, 30, 40, 50, 60, 70, 80, 90],
        "labels": [
            "00-5",  # to have it sort correctly
            "05-15",  # to have it sort correctly
            "16-17",
            "18-24",
            "25-34",
            "35-44",
            "45-54",
            "55-64",
            "65-74",
            "75-84",
            "85+",
        ],
    },
    "pgend": {
        "desc": "Gend",
        "col": "pgend",
        "vals": [1, 2, 3, 9],
        "labels": ["1_M", "2_F", "3_Other/NonBinary", "9_NA"],
    },
}


In [ ]:
writer = pd.ExcelWriter(out_filepath, engine="xlsxwriter")
workbook = writer.book
format1 = workbook.add_format({"num_format": "#,##0.0"})

In [ ]:
trip = load_trip(tour_extract_wkday_dir / "trip-assign_day.csv", hh, person)

In [ ]:
row = 0
sname = "Weekday"
d1_dict = col_dict["dpurp"]
title = "All Trips by " + d1_dict["desc"]

tab = prep_data_1d(
    trip,
    d1_dict["desc"],
    d1_dict["col"],
    "trexpfac",
    d1_dict["vals"],
    d1_dict["labels"],
)
row = write_to_excel(writer, tab, sname, title, row)
title = "Column Shares by " + d1_dict["desc"]
row = write_to_excel(writer, getSharesIdx(tab.copy()), sname, title, row)
tab2 = tab.copy()
tab2.iloc[-1, -1] = tab2.iloc[-1, 0]
title = "Column Shares 95% CI by " + d1_dict["desc"]
row = write_to_excel(writer, getSharesIdxCI95(tab.copy()), sname, title, row)
tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
_ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [ ]:
wt_cols = ["count", "trexpfac"]
wt_desc = ["(Unweighted)", "(Weighted)"]

for key in ["tod", "hh_income_broad_cat", "age", "pgend"]:  # TODO add back in raceeth
    # for key in ["tod", "raceeth", "hh_income_broad_cat", "age", "pgend"]:
    d2_dict = col_dict[key]
    row = 0
    sname = d2_dict["desc"]

    for wc, wd in zip(wt_cols, wt_desc):
        title = "All Trips by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
        tab = prep_data_2d(
            trip,
            d1_dict["col"],
            d1_dict["vals"],
            d1_dict["labels"],
            d2_dict["col"],
            d2_dict["vals"],
            d2_dict["labels"],
            wc,
        )
        row = write_to_excel(writer, tab.astype("float64"), sname, title, row)

        if wc == "count":
            tab2 = tab.copy()
        else:
            tab2.iloc[:-1, :-1] = tab.iloc[:-1, :-1]

        title = (
            "Column Shares by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
        )
        row = write_to_excel(writer, getSharesIdx(tab.copy()), sname, title, row)

        title = (
            "Column Shares 95% CI by "
            + d2_dict["desc"]
            + " and "
            + d1_dict["desc"]
            + " "
            + wd
        )
        row = write_to_excel(writer, getSharesIdxCI95(tab2.copy()), sname, title, row)
    tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
    _ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [ ]:
## Process all week trip records
trip = load_trip(tour_extract_allwk_dir / "trip-assign_day.csv", hh, person)

In [ ]:
row = 0
sname = "DOW"
d2_dict = col_dict["day"]

for wc, wd in zip(wt_cols, wt_desc):
    title = "All Trips by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
    tab = prep_data_2d(
        trip,
        d1_dict["col"],
        d1_dict["vals"],
        d1_dict["labels"],
        d2_dict["col"],
        d2_dict["vals"],
        d2_dict["labels"],
        wc,
    )
    row = write_to_excel(writer, tab.astype("float64"), sname, title, row)

    if wc == "count":
        tab2 = tab.copy()
    else:
        tab2.iloc[:-1, :-1] = tab.iloc[:-1, :-1]

    title = "Column Shares by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
    row = write_to_excel(writer, getSharesIdx(tab.copy()), sname, title, row)

    title = (
        "Column Shares 95% CI by "
        + d2_dict["desc"]
        + " and "
        + d1_dict["desc"]
        + " "
        + wd
    )
    row = write_to_excel(writer, getSharesIdxCI95(tab2.copy()), sname, title, row)
tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
_ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [ ]:
writer.close()